In [42]:
import pickle
import numpy as np
from sklearn import preprocessing
import pandas as pd

In [135]:
scalerfile = 'DataScaler.sav'
scaler = pickle.load(open(scalerfile, 'rb'))

In [136]:
le_stadium = pickle.load(open("le_stadium.pkl", 'rb'))
le_teams = preprocessing.LabelEncoder()
le_teams.classes_ = np.load("le_teams.npy",allow_pickle=True)
le_players = preprocessing.LabelEncoder()
le_players.classes_ = np.load("le_players.npy",allow_pickle=True)

In [153]:
data = pd.read_csv('21_inn2.csv')

In [154]:
data.head()

,venue,innings,batting_team,bowling_team,batsmen,bowlers
0,Narendra Modi Stadium,2,Kolkata Knight Riders,Punjab Kings,"N Rana,Shubman Gill,RA Tripathi","Mohammed Shami,Fabian Allen,A Singh"


In [155]:
X = data.values.tolist()[0]
X[4] = X[4].split(',')
X[5] = X[5].split(',')

In [156]:
X

['Narendra Modi Stadium',
 2,
 'Kolkata Knight Riders',
 'Punjab Kings',
 ['N Rana', 'Shubman Gill', 'RA Tripathi'],
 ['Mohammed Shami', 'Fabian Allen', 'A Singh']]

In [157]:
max_bat = 9
max_bowl = 6

In [158]:
    for j in range(max_bat-len(X[4])):
        X[4].extend(["_"])
    for j in range(max_bowl-len(X[5])):
        X[5].extend(["_"])        

In [159]:
T = X.copy()
T.extend(T[4])
T.pop(4)
T.extend(T[4])
T.pop(4)   

['Mohammed Shami', 'Fabian Allen', 'A Singh', '_', '_', '_']

In [160]:
X = T

In [161]:
def inpu(X):
    pc = le_players.classes_.shape[0]-1
    X_m = np.zeros(len(X))
    for i in range(len(X)):
        if (i==0):
            X_m[i] = le_stadium[X[i]]
        elif(i==2 or i==3):
            if(X[i]=="Punjab Kings"):
                X_m[i] = np.where(le_teams.classes_ == "Kings XI Punjab")[0][0]
            else:
                X_m[i] = np.where(le_teams.classes_ == X[i])[0][0]
        elif(i>3):
            if X[i] not in le_players.classes_: 
                X_m[i] = pc
                pc+=1
            else:
                X_m[i] = np.where(le_players.classes_ == X[i])[0][0]
            
    return X_m        

In [162]:
x_t = inpu(X)

In [163]:
x_t

array([ 35.,   0.,   7.,   5., 345., 520., 406., 583., 583., 583., 583.,
       583., 583., 338., 584.,  13., 583., 583., 583.])

In [164]:
xx = np.where(x_t==le_players.classes_.shape[0]-1,-1, x_t)

In [165]:
xx

array([ 35.,   0.,   7.,   5., 345., 520., 406.,  -1.,  -1.,  -1.,  -1.,
        -1.,  -1., 338., 584.,  13.,  -1.,  -1.,  -1.])

In [166]:
filename = 'RF_trail_10.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [167]:
y = int(np.ceil(loaded_model.predict(xx.reshape(-1,1).T.astype(int))[0]))

In [168]:
print(y)

49
